In [ ]:
# import libraries
import numpy as np
import pandas as pd
import requests
import zipfile
import io
from datetime import datetime
import json
import time
from pprint import pprint

In [ ]:
# import dataframes (set working directory)
df_day=pd.read_csv('data_per_day.csv',parse_dates=['date'])
df_weather=pd.read_csv('weather.csv',parse_dates=['time'],dayfirst=True)
df_covid=pd.read_csv('covid_nyc.csv',parse_dates=['date_of_interest'])

In [ ]:
df_day.head()

In [ ]:
df_weather.head()

In [ ]:
df_covid.head()

In [ ]:
# wind chill calculation
df_weather['WC']=13.12 + 0.6215*df_weather['tavg'] - 11.37*(df_weather['wspd']**0.16)+0.3965*df_weather['tavg']*(df_weather['wspd']**0.16)

In [ ]:
# enrichment
df_1=pd.merge(df_day,df_weather,left_on='date',right_on='time')
df_2=pd.merge(df_1,df_covid,left_on='date',right_on='date_of_interest',how='outer')
df_2.drop(columns='time',inplace=True)
df_2.drop(columns='date_of_interest',inplace=True)
df_final=df_2[df_2['date'].notna()]
df_final.to_csv('data_per_day_enriched.csv',index=False)

In [ ]:
# create auxiliary dataframes
aux=df_final.copy()

only20=aux[aux['date'].dt.year==2020]

only19=aux[aux['date'].dt.year==2019]
only19=only19.iloc[:,0:16]
only19['date'].dt.year

only20['daymonth']=only20['date'].dt.strftime('%d-%m')
only19['daymonth']=only19['date'].dt.strftime('%d-%m')

In [ ]:
# creating dataframe with 2020 and 2019 data per day
df_day=pd.merge(only20,only19,on='daymonth',how='outer',suffixes=['_20','_19'])
df_day.to_csv('data_per_day_combined.csv')

In [ ]:
# same thing but with syncronized week
aux['week']=aux['date'].dt.strftime('%U')
aux['dayOfWeek']=aux['date'].dt.strftime('%A')
only20=aux[aux['date'].dt.year==2020]
only19=aux[aux['date'].dt.year==2019]
only19=only19.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,-1,-2]]

In [ ]:
df_sync=pd.merge(only20,only19,on=['week','dayOfWeek'],suffixes=['_20','_19'])
df_sync.to_csv('data_per_day_sync.csv',index=False)

In [ ]:
# Create dataframe for tableau
df=pd.read_csv('september_start_end.csv',index_col=0,parse_dates=['my_dates'])
df=df[['station_id','name','latitude','longitude','year','day_of_week','Hour','trips_start','trips_end']]
df['traffic']=df['trips_start']+df['trips_end']

df_2019=df[df['year']==2019].drop('year',axis=1)
data_2019=df_2019.groupby(['station_id','name','latitude','longitude','day_of_week','Hour']).mean().reset_index()

df_2020=df[df['year']==2020].drop('year',axis=1)
data_2020=df_2020.groupby(['station_id','name','latitude','longitude','day_of_week','Hour']).mean().reset_index()

avg_traffic=pd.merge(data_2020,data_2019,on=['station_id','name','latitude','longitude','day_of_week','Hour'],
               suffixes=['_2020','_2019'],how='outer')

avg_traffic.to_csv('september_avgtraffic.csv',index=False)

In [ ]:
#join between september start_end - a
SE = pd.read_csv("september_start_end.csv")
AVG = pd.read_csv("september_avgtraffic.csv")
per_map=pd.merge( SE,AVG, on=['station_id','name','latitude','longitude','day_of_week','Hour'])
per_map.to_csv("per_map.csv")